In [1]:
import mlflow
from azureml.core import Workspace, Experiment, Environment, Datastore, Dataset, ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from IPython.display import clear_output
import time
# from ray_on_azureml.ray_on_aml import getRay
import sys
sys.path.append("../") # go to parent dir
import importlib
from src.ray_on_azureml.ray_on_aml import Ray_On_AML


In [2]:
ws = Workspace.from_config()

In [3]:
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="worker-cpu-v3")

In [10]:
ray_on_aml.shutdown()

In [4]:
run, ray = ray_on_aml.getRay()

azureml_py38


2021-12-05 17:54:08,788	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


Found existing cluster, use it.


2021-12-05 17:54:24,358	INFO worker.py:842 -- Connecting to existing Ray cluster at address: 10.0.0.11:6379
2021-12-05 17:54:24,492	WARNING worker.py:1219 -- The autoscaler failed with the following error:
Terminated with signal 15
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 543, in <module>
    monitor.run()
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 439, in run
    self._run()
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 346, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)



In [5]:
# import ray
# ray.init(dashboard_host="0.0.0.0")
ray.cluster_resources()

{'node:10.0.0.29': 1.0,
 'object_store_memory': 38201283377.0,
 'CPU': 26.0,
 'memory': 83487628905.0,
 'node:10.0.0.20': 1.0,
 'node:10.0.0.25': 1.0,
 'node:10.0.0.27': 1.0,
 'node:10.0.0.11': 1.0,
 'accelerator_type:K80': 1.0,
 'GPU': 1.0,
 'node:10.0.0.17': 1.0}

In [11]:
# import ray
# ray.init()
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
# ray.shutdown()
# client = ray.init(address =f'ray://{headnode_private_ip}:10001',allow_multiple=True  )
# with client:
# Start Ray.
# Tip: If connecting to an existing cluster, use ray.init(address="auto").
# ray.init(address ='ray://10.0.0.14:10001',ignore_reinit_error=True)
d_arr = da.from_array(np.random.randint(0, 1000, size=(256, 256)))

# The Dask scheduler submits the underlying task graph to Ray.
d_arr.mean().compute(scheduler=ray_dask_get)

# Set the scheduler to ray_dask_get in your config so you don't have to
# specify it on each compute call.
dask.config.set(scheduler=ray_dask_get)

df = dd.from_pandas(
    pd.DataFrame(
        np.random.randint(0, 10000, size=(1024, 2)), columns=["age", "grade"]),
    npartitions=2)
df.groupby(["age"]).mean().compute()

# ray.shutdown()

,grade
age,
12,3524.5
14,693.0
24,8510.0
51,6913.0
53,5590.0
...,...
9930,933.0
9944,9450.0
9952,2110.0


In [7]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
# import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # In this example, we don't change the model architecture
        # due to simplicity.
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)
# Change these values if you want the training to run quicker or slower.
EPOCH_SIZE = 512
TEST_SIZE = 256

def train(model, optimizer, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
            return
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            if batch_idx * len(data) > TEST_SIZE:
                break
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total
def train_mnist(config):
    # Data Setup
    mnist_transforms = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])

    train_loader = DataLoader(
        datasets.MNIST("~/data", train=True, download=True, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)
    test_loader = DataLoader(
        datasets.MNIST("~/data", train=False, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ConvNet()
    model.to(device)

    optimizer = optim.SGD(
        model.parameters(), lr=config["lr"], momentum=config["momentum"])
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)

        # Send the current training result back to Tune
        tune.report(mean_accuracy=acc)

        if i % 5 == 0:
            # This saves the model to the trial directory
            torch.save(model.state_dict(), "./model.pth")
search_space = {
    "lr": tune.sample_from(lambda spec: 10**(-10 * np.random.rand())),
    "momentum": tune.uniform(0.01, 0.09)
}

# Uncomment this to enable distributed execution
# ray.shutdown()
# ray.init(address="auto",ignore_reinit_error=True)
# ray.init(address =f'ray://{headnode_private_ip}:10001',allow_multiple=True,ignore_reinit_error=True )
# Download the dataset first
datasets.MNIST("~/data", train=True, download=True)

analysis = tune.run(train_mnist, config=search_space)


2021-12-05 04:57:52,298	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-12-05 04:57:52,311	INFO logger.py:605 -- pip install "ray[tune]" to see TensorBoard files.
2021-12-05 04:57:52,311	WARNING callback.py:114 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2021-12-05 04:57:52,330	WARNING tune.py:570 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,lr,momentum
train_mnist_e6d1f_00000,PENDING,,1.69244e-07,0.0674072


(ImplicitFunc pid=21852) /anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
(ImplicitFunc pid=21852)   return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Result for train_mnist_e6d1f_00000:
  date: 2021-12-05_04-57-53
  done: false
  experiment_id: dc6cebb9cdd14cd4aa515c02c9d89af4
  hostname: nc6v0
  iterations_since_restore: 1
  mean_accuracy: 0.11875
  node_ip: 10.0.0.11
  pid: 21852
  time_since_restore: 0.25304245948791504
  time_this_iter_s: 0.25304245948791504
  time_total_s: 0.25304245948791504
  timestamp: 1638680273
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: e6d1f_00000
  
Result for train_mnist_e6d1f_00000:
  date: 2021-12-05_04-57-55
  done: true
  experiment_id: dc6cebb9cdd14cd4aa515c02c9d89af4
  experiment_tag: 0_lr=1.6924e-07,momentum=0.067407
  hostname: nc6v0
  iterations_since_restore: 10
  mean_accuracy: 0.090625
  node_ip: 10.0.0.11
  pid: 21852
  time_since_restore: 2.0839645862579346
  time_this_iter_s: 0.20478534698486328
  time_total_s: 2.0839645862579346
  timestamp: 1638680275
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: e6d1f_00000
  


Trial name,status,loc,lr,momentum,acc,iter,total time (s)
train_mnist_e6d1f_00000,TERMINATED,10.0.0.11:21852,1.69244e-07,0.0674072,0.090625,10,2.08396


2021-12-05 04:57:55,697	INFO tune.py:626 -- Total run time: 3.40 seconds (3.25 seconds for the tuning loop).


In [6]:
 import sklearn.datasets
 import sklearn.metrics
 from sklearn.model_selection import train_test_split
 import xgboost as xgb

 from ray import tune


 def train_breast_cancer(config):
     # Load dataset
     data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
     # Split into train and test set
     train_x, test_x, train_y, test_y = train_test_split(
         data, labels, test_size=0.25)
     # Build input matrices for XGBoost
     train_set = xgb.DMatrix(train_x, label=train_y)
     test_set = xgb.DMatrix(test_x, label=test_y)
     # Train the classifier
     results = {}
     xgb.train(
         config,
         train_set,
         evals=[(test_set, "eval")],
         evals_result=results,
         verbose_eval=False)
     # Return prediction accuracy
     accuracy = 1. - results["eval"]["error"][-1]
     tune.report(mean_accuracy=accuracy, done=True)


 config = {
     "objective": "binary:logistic",
     "eval_metric": ["logloss", "error"],
     "max_depth": tune.randint(1, 9),
     "min_child_weight": tune.choice([1, 2, 3]),
     "subsample": tune.uniform(0.5, 1.0),
     "eta": tune.loguniform(1e-4, 1e-1)
 }
 analysis = tune.run(
     train_breast_cancer,
     resources_per_trial={"cpu": 1},
     config=config,
     num_samples=10)


2021-12-05 05:35:20,237	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-12-05 05:35:20,242	INFO logger.py:605 -- pip install "ray[tune]" to see TensorBoard files.
2021-12-05 05:35:20,242	WARNING callback.py:114 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2021-12-05 05:35:20,259	WARNING tune.py:570 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,eta,max_depth,min_child_weight,subsample
train_breast_cancer_22b10_00000,PENDING,,0.000115884,3,1,0.820534
train_breast_cancer_22b10_00001,PENDING,,0.000159302,2,3,0.800876
train_breast_cancer_22b10_00002,PENDING,,0.0925686,3,3,0.576718
train_breast_cancer_22b10_00003,PENDING,,0.000185896,6,2,0.610289
train_breast_cancer_22b10_00004,PENDING,,0.00243399,3,3,0.554835
train_breast_cancer_22b10_00005,PENDING,,0.00398388,3,3,0.651541
train_breast_cancer_22b10_00006,PENDING,,0.0628726,8,2,0.920078
train_breast_cancer_22b10_00007,PENDING,,0.000289861,6,3,0.689321
train_breast_cancer_22b10_00008,PENDING,,0.000293346,5,3,0.997516
train_breast_cancer_22b10_00009,PENDING,,0.000158485,8,2,0.500692


Result for train_breast_cancer_22b10_00000:
  date: 2021-12-05_05-35-21
  done: true
  experiment_id: 286b920c722a4779ab42009884ed4c3e
  hostname: 71bb804dc8084a5e8ad9b06b8cb81bbb000001
  iterations_since_restore: 1
  mean_accuracy: 0.951049
  node_ip: 10.0.0.17
  pid: 227
  time_since_restore: 0.08456587791442871
  time_this_iter_s: 0.08456587791442871
  time_total_s: 0.08456587791442871
  timestamp: 1638682521
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 22b10_00000
  
Result for train_breast_cancer_22b10_00003:
  date: 2021-12-05_05-35-22
  done: true
  experiment_id: cb7bfb305c8a4595a065a1ab188e5d20
  hostname: 71bb804dc8084a5e8ad9b06b8cb81bbb00000I
  iterations_since_restore: 1
  mean_accuracy: 0.965035
  node_ip: 10.0.0.27
  pid: 216
  time_since_restore: 0.11986041069030762
  time_this_iter_s: 0.11986041069030762
  time_total_s: 0.11986041069030762
  timestamp: 1638682522
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 22b10_00003
  
Result f

Trial name,status,loc,eta,max_depth,min_child_weight,subsample,acc,iter,total time (s)
train_breast_cancer_22b10_00000,TERMINATED,10.0.0.17:227,0.000115884,3,1,0.820534,0.951049,1,0.0845659
train_breast_cancer_22b10_00001,TERMINATED,10.0.0.29:233,0.000159302,2,3,0.800876,0.923077,1,0.389669
train_breast_cancer_22b10_00002,TERMINATED,10.0.0.20:231,0.0925686,3,3,0.576718,0.93007,1,0.500888
train_breast_cancer_22b10_00003,TERMINATED,10.0.0.27:216,0.000185896,6,2,0.610289,0.965035,1,0.11986
train_breast_cancer_22b10_00004,TERMINATED,10.0.0.11:29482,0.00243399,3,3,0.554835,0.944056,1,1.33129
train_breast_cancer_22b10_00005,TERMINATED,10.0.0.25:257,0.00398388,3,3,0.651541,0.944056,1,0.102623
train_breast_cancer_22b10_00006,TERMINATED,10.0.0.20:230,0.0628726,8,2,0.920078,0.958042,1,0.4134
train_breast_cancer_22b10_00007,TERMINATED,10.0.0.11:29483,0.000289861,6,3,0.689321,0.951049,1,1.25316
train_breast_cancer_22b10_00008,TERMINATED,10.0.0.29:234,0.000293346,5,3,0.997516,0.951049,1,0.335507
train_breast_cancer_22b10_00009,TERMINATED,10.0.0.11:29481,0.000158485,8,2,0.500692,0.965035,1,1.35481


2021-12-05 05:35:24,128	INFO tune.py:626 -- Total run time: 3.89 seconds (3.74 seconds for the tuning loop).


In [7]:
#dask

# import ray
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import dask
import glob
import joblib

import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt

from datetime import datetime

from azureml.core import Workspace, Dataset, Model
data_url = 'https://azureopendatastorage.blob.core.windows.net/isdweatherdatacontainer/ISDWeather'



In [8]:
storage_options = {'account_name': 'azureopendatastorage'}
ddf = dd.read_parquet('az://isdweatherdatacontainer/ISDWeather/year=2019/month=2', storage_options=storage_options)


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [9]:
from adlfs import AzureBlobFileSystem,AzureDatalakeFileSystem
sas_token='sp=rle&st=2021-11-27T19:26:49Z&se=2022-04-22T02:26:49Z&spr=https&sv=2020-08-04&sr=c&sig=gpYJ3LofkkjkeWkrMDTY1Ge6NsOCghbUR9CQs65HCvU%3D'
account_key='ak7nSgU/iRc5XCvXjaM2UwX18ybS9WO1BsE4Vm42XLmiZvBm1kCyZQD3USeCCOiwusWkDaq0jcRF1JpPHiAMhw=='
account_name="adlsgen7"
abfs = AzureBlobFileSystem(account_name="adlsgen7",account_key=account_key,  container_name="mltraining")


In [25]:
# import dask.dataframe as dd

# storage_options={'account_name': account_name, 'account_key': account_key}

# # ddf = dd.read_csv('abfs://{CONTAINER}/{FOLDER}/*.csv', storage_options=storage_options)
# ddf = dd.read_parquet('az://mltraining/ISDWeatherDelta/year=2008/month=1/*.parquet', storage_options=storage_options)


In [10]:

data = ray.data.read_parquet("az://mltraining/ISDWeatherDelta/year2008/month", filesystem=abfs)

NotADirectoryError: mltraining/ISDWeatherDelta/year2008/month

In [57]:
data.count()

8103070

In [28]:
dask.config.set(scheduler=ray_dask_get)


In [11]:
for year in range(2019, 2020+1):
    ds = Dataset.File.from_files(f'{data_url}/year={year}/month=*/*.parquet', validate=False)
    print('Downloading...')
    %time ds.download(f'/tmp/data/isd/year={year}', overwrite=True)


Downloading...
CPU times: user 5.35 s, sys: 4.77 s, total: 10.1 s
Wall time: 35.1 s
Downloading...
CPU times: user 6.06 s, sys: 4.88 s, total: 10.9 s
Wall time: 56 s


In [12]:
files = glob.glob('/tmp/data/isd/year=*/month=*/*.parquet')
len(files)

192

In [13]:
dfs = [dask.delayed(pd.read_parquet)(file) for file in files]
dfs[-5:]
df = dd.from_delayed(dfs)
%time df.describe().compute()
# CPU times: user 20.5 s, sys: 4.63 s, total: 25.2 s
# Wall time: 2min 38s


RayTaskError(FileNotFoundError): [36mray::dask:make_meta-04803ab8-eacd-4c1c-8dba-c12bef057b2d[39m (pid=207, ip=10.0.0.17)
  At least one of the input arguments for this task could not be computed:
ray.exceptions.RayTaskError: [36mray::dask:read_parquet-568d98c5-6b4e-4819-b9bf-b4760b82036f[39m (pid=207, ip=10.0.0.17)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/util/dask/scheduler.py", line 350, in dask_task_wrapper
  File "/azureml-envs/azureml_8afa3af27adc7f1a69c133d28398582a/lib/python3.8/site-packages/pandas/io/parquet.py", line 495, in read_parquet
    return impl.read(
  File "/azureml-envs/azureml_8afa3af27adc7f1a69c133d28398582a/lib/python3.8/site-packages/pandas/io/parquet.py", line 232, in read
    path_or_handle, handles, kwargs["filesystem"] = _get_path_or_handle(
  File "/azureml-envs/azureml_8afa3af27adc7f1a69c133d28398582a/lib/python3.8/site-packages/pandas/io/parquet.py", line 101, in _get_path_or_handle
    handles = get_handle(
  File "/azure

In [40]:
%time places = df.groupby(df.index)[['longitude', 'latitude', 'year']].mean().compute()


KeyboardInterrupt: 

In [11]:
import ray
import raydp
import os
# ray.shutdown()
# ray.init()
os.environ["PYSPARK_PYTHON"]="/anaconda/envs/azureml_py38/bin/python3"

# ray.init(address ='ray://10.0.0.11:6379')
spark = raydp.init_spark(
  app_name = "example",
  num_executors = 2,
  executor_cores = 1,
  executor_memory = "1gb"
)

# data =spark.read.format("csv").option("header", True).load("wasbs://ojsales-simulatedcontainer@azureopendatastorage.blob.core.windows.net/oj_sales_data/Store10*.csv")


# # normal data processesing with Spark
# df = spark.createDataFrame([('look',), ('spark',), ('tutorial',), ('spark',), ('look', ), ('python', )], ['word'])
# df.show()
# word_count = df.groupBy('word').count()
# word_count.show()
import pandas as pd

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

# Declare the function and create the UDF
def multiply_func(a: pd.Series, b: pd.Series) -> pd.Series:
    return a * b

multiply = pandas_udf(multiply_func, returnType=LongType())

# The function for a pandas_udf should be able to execute with local Pandas data
x = pd.Series([1, 2, 3])
print(multiply_func(x, x))
# 0    1
# 1    4
# 2    9
# dtype: int64

# Create a Spark DataFrame, 'spark' is an existing SparkSession
df = spark.createDataFrame(pd.DataFrame(x, columns=["x"]))

# Execute function as a Spark vectorized UDF
df.select(multiply(col("x"), col("x"))).show()
# +-------------------+
# |multiply_func(x, x)|
# +-------------------+
# |                  1|
# |                  4|
# |                  9|
# +-------------------+


# stop the spark cluster
raydp.stop_spark()


2021-12-05 05:35:59,006 WARN Utils [Thread-1]: Your hostname, nc6v0 resolves to a loopback address: 127.0.0.1; using 10.0.0.11 instead (on interface eth0)
2021-12-05 05:35:59,010 WARN Utils [Thread-1]: Set SPARK_LOCAL_IP if you need to bind to another address


2021-12-05 05:35:59,214 WARN NativeCodeLoader [Thread-1]: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-12-05 05:35:59,294 INFO SecurityManager [Thread-1]: Changing view acls to: azureuser
2021-12-05 05:35:59,295 INFO SecurityManager [Thread-1]: Changing modify acls to: azureuser
2021-12-05 05:35:59,295 INFO SecurityManager [Thread-1]: Changing view acls groups to: 
2021-12-05 05:35:59,296 INFO SecurityManager [Thread-1]: Changing modify acls groups to: 
2021-12-05 05:35:59,296 INFO SecurityManager [Thread-1]: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(azureuser); groups with view permissions: Set(); users  with modify permissions: Set(azureuser); groups with modify permissions: Set()
2021-12-05 05:35:59,569 INFO Utils [Thread-1]: Successfully started service 'RAY_RPC_ENV' on port 39967.


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/jars/ray_dist.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


2021-12-05 05:36:02,814 INFO RayAppMaster$RayAppMasterEndpoint [dispatcher-event-loop-1]: Registering app example
2021-12-05 05:36:02,817 INFO RayAppMaster$RayAppMasterEndpoint [dispatcher-event-loop-1]: Registered app example with ID app-20211205053602-0000


(raylet, ip=10.0.0.20) [2021-12-05 05:36:02,985 C 157 157] worker_pool.cc:1236:  Check failed: state != states_by_lang_.end() Required Language isn't supported: JAVA
(raylet, ip=10.0.0.20) *** StackTrace Information ***
(raylet, ip=10.0.0.20)     ray::SpdLogMessage::Flush()
(raylet, ip=10.0.0.20)     ray::RayLog::~RayLog()
(raylet, ip=10.0.0.20)     ray::raylet::WorkerPool::GetStateForLanguage()
(raylet, ip=10.0.0.20)     ray::raylet::WorkerPool::PopWorker()
(raylet, ip=10.0.0.20)     ray::raylet::ClusterTaskManager::DispatchScheduledTasksToWorkers()
(raylet, ip=10.0.0.20)     ray::raylet::ClusterTaskManager::QueueAndScheduleTask()
(raylet, ip=10.0.0.20)     ray::raylet::NodeManager::HandleRequestWorkerLease()
(raylet, ip=10.0.0.20)     std::_Function_handler<>::_M_invoke()
(raylet, ip=10.0.0.20)     boost::asio::detail::completion_handler<>::do_complete()
(raylet, ip=10.0.0.20)     boost::asio::detail::scheduler::do_run_one()
(raylet, ip=10.0.0.20)     boost::asio::detail::scheduler::

0    1
1    4
2    9
dtype: int64


ERROR:root:Exception while sending command.                         (0 + 0) / 1]
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=133>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/py4j/clientserver.py", line 503, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/py4j/clientserver.py", line 475, i

Py4JError: An error occurred while calling o107.showString

2021-12-05 05:36:32,662	WARNING worker.py:1245 -- The node with node id: 2e302da25deb2e6ec2b45913710812b94636e3f2604b13c93f623695 and ip: 10.0.0.20 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a raylet crashes unexpectedly or has lagging heartbeats.
2021-12-05 06:15:31,724	WARNING worker.py:1245 -- The node with node id: f9377cbb25de1a81861f67bbdd0192e75c2f04901bb5eaff3a643221 and ip: 10.0.0.29 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a raylet crashes unexpectedly or has lagging heartbeats.
2021-12-05 06:15:31,724	WARNING worker.py:1245 -- The node with node id: e4222486f8a5eaa3dbb7aa75ba096e6899736afb704bfb9d38b01330 and ip: 10.0.0.27 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a raylet crashes unexpectedly or has lagging heartbeats.
2021-12-05 06:15:31,725	WARNING worker.py:1245 -- The node with node id: bb3